## Adding a Spotify Tool - Musically You!

This version of the notebook introduces a Spotify tool that can query your listening history from Spotify to extend the domain of questions the chatbot can answer to include your musical tastes.

Unfortunately, it's a bit of PITA to get acess and refresh tokens for Spotify.  The process requires connecting to an authentication end point while logged in to Spotify and then processing a callback.  To make this easier, instructions along with a small app that can be deployed to HuggingFace Spaces using Gradio are included at the end of this notebook. 

## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [23]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [24]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

In [26]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [27]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [28]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
# For Spotify access token and refresh token
import base64
import time
import hashlib
import secrets
import urllib.parse

spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

if spotify_client_id:
    print(f"Spotify client ID found and starts with {spotify_client_id[:4]}")
else:
    print("Spotify client ID not found")

if spotify_client_secret:
    print(f"Spotify client secret found and starts with {spotify_client_secret[:4]}")
else:
    print("Spotify client secret not found")

spotify_access_token = os.getenv("SPOTIFY_ACCESS_TOKEN")
spotify_refresh_token = os.getenv("SPOTIFY_REFRESH_TOKEN")

if spotify_access_token and spotify_refresh_token:
    # Set expiry to past to force refresh on first use
    spotify_token_expiry = time.time() - 60
    print("Spotify tokens loaded from environment!")
    print(f"Access token preview: {spotify_access_token[:20]}...")
    print(f"Refresh token preview: {spotify_refresh_token[:20]}...")
else:
    print("No Spotify tokens found in environment. Run spotify_flask_auth.py to get them.")

In [30]:
def get_spotify_access_token():
    global spotify_access_token, spotify_refresh_token, spotify_token_expiry
    
    # Check if we have a valid cached token
    if spotify_access_token and time.time() < spotify_token_expiry:
        return spotify_access_token
    

    auth_url = "https://accounts.spotify.com/api/token"
    
    credentials = f"{spotify_client_id}:{spotify_client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "refresh_token",
        "refresh_token": spotify_refresh_token
    }
    
    response = requests.post(auth_url, headers=headers, data=data)
    
    if response.status_code == 200:
        token_data = response.json()
        spotify_access_token = token_data["access_token"]
        # Update refresh token if a new one is provided
        if "refresh_token" in token_data:
            spotify_refresh_token = token_data["refresh_token"]
        # Set expiry time with a buffer
        spotify_token_expiry = time.time() + token_data["expires_in"] - 300
        return spotify_access_token
    else:
        print(f"Failed to refresh Spotify access token: {response.status_code}")
        print(f"Response: {response.text}")
        return None


In [31]:
def get_user_top_items(item_type="artists", time_range="medium_term", limit=10):
    """
    Get the user's top artists or tracks from Spotify.
    
    Args:
        item_type: 'artists' or 'tracks'
        time_range: 'short_term' (4 weeks), 'medium_term' (6 months), 'long_term' (several years)
        limit: Number of items to return (1-50)
    
    Returns:
        Dictionary with top items data
    """
    token = get_spotify_access_token()
    if not token:
        return {"error": "Failed to get Spotify access token"}
    
    # Make API request
    url = f"https://api.spotify.com/v1/me/top/{item_type}"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "time_range": time_range,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        formatted_items = []
        for idx, item in enumerate(data.get("items", []), 1):
            if item_type == "artists":
                formatted_items.append({
                    "rank": idx,
                    "name": item["name"],
                    "genres": item.get("genres", []),
                    "popularity": item.get("popularity", 0),
                    "spotify_url": item["external_urls"]["spotify"]
                })
            else:  # tracks
                formatted_items.append({
                    "rank": idx,
                    "name": item["name"],
                    "artist": item["artists"][0]["name"] if item.get("artists") else "Unknown",
                    "album": item["album"]["name"] if item.get("album") else "Unknown",
                    "popularity": item.get("popularity", 0),
                    "spotify_url": item["external_urls"]["spotify"]
                })
        
        return {
            "item_type": item_type,
            "time_range": time_range,
            "count": len(formatted_items),
            "items": formatted_items
        }
    else:
        return {"error": f"Failed to get top items: {response.status_code} - {response.text}"}

In [ ]:
# lets test the tool
get_user_top_items(item_type="artists", time_range="medium_term", limit=3)

In [33]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [34]:
get_user_top_items_json = {
    "name": "get_user_top_items",
    "description": "Get the user's top artists or tracks from Spotify based on their listening history",
    "parameters": {
        "type": "object",
        "properties": {
            "item_type": {
                "type": "string",
                "description": "Type of items to retrieve: 'artists' or 'tracks'",
                "enum": ["artists", "tracks"]
            },
            "time_range": {
                "type": "string", 
                "description": "Time range for the data: 'short_term' (4 weeks), 'medium_term' (6 months), or 'long_term' (several years)",
                "enum": ["short_term", "medium_term", "long_term"]
            },
            "limit": {
                "type": "integer",
                "description": "Number of items to return (1-50)",
                "minimum": 1,
                "maximum": 50
            }
        },
        "required": ["item_type", "time_range", "limit"],
        "additionalProperties": False
    }
}

In [35]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [36]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json},
        {"type": "function", "function": get_user_top_items_json}]

In [ ]:
tools

In [38]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        elif tool_name == "get_user_top_items":
            result = get_user_top_items(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

In [40]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [41]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [42]:
# We've added a "If they ask you about your tastes in music you can use your get_user_top_items tool..." 

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If they ask you about your tastes in music you can use your get_user_top_items tool to get information about your top artists and tracks. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [43]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


## Spotify API Setup Instructions

To use the Spotify tool in this notebook, you need will need to undergo a one-time setup process to obtain access and refresh tokens from Spotify.  This involve the following steps:

1. **Create a Spotify App**:
   - Go to https://developer.spotify.com/dashboard
   - Click "Create app"
   - Fill in the app details
   - Set Redirect URI to: `https://your-username-your-space-name.hf.space/callback` (replace with your actual HuggingFace Space URL)
   - Save your Client ID and Client Secret

2. **Add to your `.env` file**:
   ```
   SPOTIFY_CLIENT_ID=your_client_id_here
   SPOTIFY_CLIENT_SECRET=your_client_secret_here
   ```

3. **Deploy and authenticate**:
   - Deploy the authentication app from the **Flask Authentication App for Spotify** cell below to HuggingFace Spaces
   - Visit your deployed app and click "Authorize with Spotify"
   - After authorizing, copy the tokens displayed
   - ONCE YOU HAVE HAVE OBTAINED YOUR ACCESS AND REFRESH TOKESNS YOU CAN DELETE THIS DEPLOYMENT

4. **Add tokens to .env and reload**:
   ```
   SPOTIFY_ACCESS_TOKEN=your_access_token
   SPOTIFY_REFRESH_TOKEN=your_refresh_token
   ```
   Then run `load_dotenv(override=True)`



## Flask Authentication App for Spotify

Deploy this code as `spotify_flask_auth.py` to HuggingFace Spaces using Gradio following the steps as above fro
app.py.  You need the following:
1. SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET defined in your environment
2. Need a requirements.txt with Flask listed as a dependency (no other dependencies are needed)

```python
from flask import Flask, request, redirect, render_template_string
import requests
import base64
import os
from dotenv import load_dotenv
import urllib.parse
import secrets
import string

load_dotenv(override=True)

app = Flask(__name__)
app.secret_key = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(32))

CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

REDIRECT_URI = f"https://{os.getenv('SPACE_HOST')}/callback"
SCOPE = "user-top-read"
tokens = {}

# HTML template for the home page
HOME_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Spotify OAuth Helper</title>
</head>
<body>
    {% if has_credentials %}
        <div style="margin-bottom: 20px;">
            <p>Make sure to add this redirect URI to your Spotify app settings:</p>
            <code>{{ redirect_uri }}</code>
        </div>
        <button onclick="window.open('https://accounts.spotify.com/authorize?client_id={{ client_id }}&response_type=code&redirect_uri={{ redirect_uri | urlencode }}&scope={{ scope | urlencode }}&show_dialog=true', '_blank')">Authorize with Spotify</button>
    {% else %}
        <div>Missing Spotify credentials in .env file</div>
    {% endif %}
</body>
</html>
"""

SUCCESS_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Spotify OAuth - Success!</title>
</head>
<body>
    <p>Authorization Complete</p>
    <h3>Add to your .env file:</h3>
    <pre>SPOTIFY_ACCESS_TOKEN={{ access_token }}
SPOTIFY_REFRESH_TOKEN={{ refresh_token }}</pre>
</body>
</html>
"""

@app.route('/')
def home():
    error = request.args.get('error')
    has_credentials = CLIENT_ID and CLIENT_SECRET
    return render_template_string(HOME_TEMPLATE, error=error, has_credentials=has_credentials, redirect_uri=REDIRECT_URI, client_id=CLIENT_ID, scope=SCOPE)

@app.route('/authorize')
def authorize():
    """Redirect to Spotify authorization"""
    if not CLIENT_ID:
        return redirect('/?error=Missing SPOTIFY_CLIENT_ID')
    
    auth_url = "https://accounts.spotify.com/authorize"
    params = {
        "client_id": CLIENT_ID,
        "response_type": "code",
        "redirect_uri": REDIRECT_URI,
        "scope": SCOPE,
        "show_dialog": "true"
    }
    
    url = f"{auth_url}?{urllib.parse.urlencode(params)}"
    return redirect(url)

@app.route('/callback')
def callback():
    """Handle the OAuth callback"""
    error = request.args.get('error')
    if error:
        return redirect(f'/?error=Authorization failed: {error}')
    
    code = request.args.get('code')
    if not code:
        return redirect('/?error=No authorization code received')
    
    # Exchange code for tokens
    token_url = "https://accounts.spotify.com/api/token"
    
    credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": REDIRECT_URI
    }
    
    response = requests.post(token_url, headers=headers, data=data)
    
    if response.status_code == 200:
        token_data = response.json()
        tokens['access_token'] = token_data['access_token']
        tokens['refresh_token'] = token_data['refresh_token']
        
        return render_template_string(
            SUCCESS_TEMPLATE,
            access_token=token_data['access_token'],
            refresh_token=token_data['refresh_token']
        )
    else:
        error_msg = response.json().get('error_description', 'Unknown error')
        return redirect(f'/?error=Token exchange failed: {error_msg}')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=7860)
```

**Deployment Instructions:**
1. You will need to provide `SPOTIFY_CLIENT_ID` and `SPOTIFY_CLIENT_SECRET` as secrets in HuggingFace Spaces
2. Create a `requirements.txt` file with a single entry: `Flask`
3. Deploy to HuggingFace Spaces using the instructions in the deployment section below

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>